# 動作チェックサンプル
openai apiの接続テストと、supabase（データベースサービス）への接続とデータの読み出しテストを行います。

https://openai.com/blog/openai-api

https://supabase.com/

In [ ]:
# Pythonライブラリインストール
!python --version
!pip install psycopg2-binary
!pip install python-dotenv
!pip install anytree
!pip install --upgrade openai
!pip install openai[datalib]

## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [35]:
# 環境変数
import os
from dotenv import load_dotenv
load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

print('環境変数読み込み完了')

環境変数読み込み完了


# OPENAI API接続テスト
modelにgpt-4を用いる場合はコードを変更してください。

In [28]:
# OPENAI API接続テスト
import openai
openai.api_key = openai_api_key

completion = openai.ChatCompletion.create(
  #model="gpt-4",
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "関西弁で答えてください。"},
    {"role": "system", "content": "文の最後に「知らんけど。」を付けてください。"},
    {"role": "user", "content": "日本の首都はどこですか?"}
  ]
)

print(completion.choices[0].message.content)
print(completion)

大阪かと思ったけど、実は東京やで。知らんけど。
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\u5927\u962a\u304b\u3068\u601d\u3063\u305f\u3051\u3069\u3001\u5b9f\u306f\u6771\u4eac\u3084\u3067\u3002\u77e5\u3089\u3093\u3051\u3069\u3002",
        "role": "assistant"
      }
    }
  ],
  "created": 1682948330,
  "id": "chatcmpl-7BO1qJ92CLnjMVw5dWpUj5vyg1mR6",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 59,
    "total_tokens": 85
  }
}


# Embedding テスト

In [31]:
import openai
openai.api_key = openai_api_key

def get_embedding(model: str, text: str) -> list[float]:
    result = openai.Embedding.create(
      model = model,
      input = text
    )
    return result['data'][0]['embedding']

print(get_embedding('text-embedding-ada-002', 'This is a test'))

[-0.008171741850674152, -0.0037126264069229364, -0.0005730789271183312, -0.00573078915476799, -0.02456146851181984, 0.01606602780520916, -0.014824082143604755, -0.00460444949567318, -0.000950452231336385, -0.03453667461872101, 0.01601317897439003, 0.017294762656092644, -0.009083383716642857, 0.0022246032021939754, 0.008634168654680252, -2.691790996323107e-06, 0.02448219619691372, 0.0006089995731599629, 0.008264227770268917, -0.007511132396757603, 0.005463242065161467, 0.007577193435281515, -0.011488002724945545, 0.02479928731918335, -0.028326943516731262, -0.02324024960398674, 0.0034615949261933565, -0.03527655825018883, 0.019382288679480553, -0.009935569949448109, 0.02186618186533451, -0.01738724671304226, 0.0016746455803513527, -0.03628068417310715, 0.0007448374526575208, -0.012617644853889942, -0.010450845584273338, -0.017294762656092644, 0.008039619773626328, -0.010880242101848125, 0.009096595458686352, 0.016660576686263084, 0.0071345847100019455, -0.0213641170412302, -0.0048059355

# supabase接続テスト
supabaseから専用クライアントが公開されていますが、今回はopenai apiを使ってSQLを発行させ、それを実行させたいので、PostgreSQL形式でクエリを実行します。

In [43]:
import psycopg2
from psycopg2.extras import DictCursor

# Connect to the database
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_pass,
    host=db_host,
    port=db_port
)
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor(cursor_factory=DictCursor)


cur.execute('SELECT id,name,acronym FROM structures ORDER BY id ASC;')
rows = cur.fetchall()

for index, row in enumerate(rows):
    print(row)


[1, 'Tuberomammillary nucleus, ventral part', 'TMv']
[2, 'Primary somatosensory area, mouth, layer 6b', 'SSp-m6b']
[3, 'secondary fissure', 'sec']
[4, 'Inferior colliculus', 'IC']
[6, 'internal capsule', 'int']
[7, 'Principal sensory nucleus of the trigeminal', 'PSV']
[8, 'Basic cell groups and regions', 'grey']
[9, 'Primary somatosensory area, trunk, layer 6a', 'SSp-tr6a']
[10, 'Superior colliculus, motor related, intermediate gray layer', 'SCig']
[11, 'posterolateral fissure', 'plf']
[12, 'Interfascicular nucleus raphe', 'IF']
[14, 'internal medullary lamina of the thalamus', 'im']
[15, 'Parataenial nucleus', 'PT']
[16, 'Layer 6b, isocortex', '6b']
[17, 'Superior colliculus, motor related, intermediate white layer', 'SCiw']
[18, 'nodular fissure', 'nf']
[19, 'Induseum griseum', 'IG']
[20, 'Entorhinal area, lateral part, layer 2', 'ENTl2']
[21, 'lateral olfactory tract, general', 'lotg']
[22, 'Posterior parietal association areas', 'PTLp']
[23, 'Anterior amygdalar area', 'AAA']
[25, '

s